In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject



Il te faut d'abord extraire tous les skills dans les CV, a toi de voir le procédé le plus adequat. 

Tu as la possibilité entre opencv et les regulars expressions.


In [2]:
df = pd.read_csv('cv.csv')

In [3]:
df

,nom,skills
0,Alice Dupont.pdf,"['ABC', 'Paris']"
1,Gael Adebayo.pdf,[]
2,Jean Dupont.pdf,"['5 ans', 'ABC', 'Paris']"
3,Pierre Martin.pdf,"['Bank', 'Paris']"
4,Sarah Smith.pdf,"['ABC', 'Paris']"


In [4]:
df.dtypes

nom       object
skills    object
dtype: object

In [5]:
competences_recherches = ['5 ans','Bank','ABC','Paris']

In [6]:
# Ajouter une nouvelle ligne avec les compétences recherchées
new_row = {'nom': 'emploi', 'skills': competences_recherches}
df = df.append(new_row, ignore_index=True)

# Afficher le DataFrame mis à jour
df

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12060\228371693.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


,nom,skills
0,Alice Dupont.pdf,"['ABC', 'Paris']"
1,Gael Adebayo.pdf,[]
2,Jean Dupont.pdf,"['5 ans', 'ABC', 'Paris']"
3,Pierre Martin.pdf,"['Bank', 'Paris']"
4,Sarah Smith.pdf,"['ABC', 'Paris']"
5,emploi,"[5 ans, Bank, ABC, Paris]"


In [7]:
def clean_skills(skills_list):
    # Convertir la liste en une chaîne de caractères
    skills_str = ', '.join(skills_list)
    # Appliquer la méthode str.replace pour enlever les caractères indésirables
    skills_str = skills_str.replace('[', '').replace(']', '').replace('\'', '').replace('\"', '').replace(',', '')
    return skills_str

# Appliquer la fonction clean_skills à la ligne qui a "nom" égal à "emploi"
df.loc[df['nom'] == 'emploi', 'skills'] = df.loc[df['nom'] == 'emploi', 'skills'].apply(clean_skills)

In [8]:
df['skills'] = df['skills'].str.replace('[\[\]\'\",]', '')

df

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12060\2560733127.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['skills'] = df['skills'].str.replace('[\[\]\'\",]', '')


,nom,skills
0,Alice Dupont.pdf,ABC Paris
1,Gael Adebayo.pdf,
2,Jean Dupont.pdf,5 ans ABC Paris
3,Pierre Martin.pdf,Bank Paris
4,Sarah Smith.pdf,ABC Paris
5,emploi,5 ans Bank ABC Paris


Là je considère que tu l'as déjà fait, du auras une colonne qui te permet de faire une jointure de tous les skills chez chaque profil

Maintenat moi j'ai une colonne qui donne le nom (colonne titre) mais en plus j'ai des colonnes pré faites pour des fonctions cibles de lentreprise. par exemple un profil dev parfait et les skills associé

Pour traiter tous celà de manière vriament basique on peu utiliser CounterVectorizer

In [9]:
#Créer un objet CV
cv=CountVectorizer()
count_matrix=cv.fit_transform(df["skills"])

In [10]:
cosine_sim=cosine_similarity(count_matrix)

In [11]:
index = df[df['nom']=='emploi'].index[0]
index

5

In [12]:
# Obtenir les scores de similarité cosinus pour le profil
similarity_scores = list(enumerate(cosine_sim[index]))
similarity_scores

[(0, 0.7071067811865475),
 (1, 0.0),
 (2, 0.8660254037844388),
 (3, 0.7071067811865475),
 (4, 0.7071067811865475),
 (5, 1.0)]

In [13]:
# Trier les scores de similarité cosinus par ordre décroissant
sorted_scores = sorted(similarity_scores, key=lambda x:x[1], reverse=True)
sorted_scores

[(5, 1.0),
 (2, 0.8660254037844388),
 (0, 0.7071067811865475),
 (3, 0.7071067811865475),
 (4, 0.7071067811865475),
 (1, 0.0)]

In [14]:
# Créer une liste des 5 meilleurs profils avec leurs scores de similarité cosinus
top_5_profiles = [(df.iloc[sorted_scores[i][0]]['nom'], df.iloc[sorted_scores[i][0]]['skills'], sorted_scores[i][1]) for i in range(6)]
top_5_profiles

[('emploi', '5 ans Bank ABC Paris', 1.0),
 ('Jean Dupont.pdf', '5 ans ABC Paris', 0.8660254037844388),
 ('Alice Dupont.pdf', 'ABC Paris', 0.7071067811865475),
 ('Pierre Martin.pdf', 'Bank Paris', 0.7071067811865475),
 ('Sarah Smith.pdf', 'ABC Paris', 0.7071067811865475),
 ('Gael Adebayo.pdf', '', 0.0)]

In [15]:
# Créer un dataframe à partir de la liste des 5 meilleurs profils
df_top_5 = pd.DataFrame(top_5_profiles, columns=['titre', 'skills', 'similarité'])
df_top_5

,titre,skills,similarité
0,emploi,5 ans Bank ABC Paris,1.000000
1,Jean Dupont.pdf,5 ans ABC Paris,0.866025
2,Alice Dupont.pdf,ABC Paris,0.707107
3,Pierre Martin.pdf,Bank Paris,0.707107
4,Sarah Smith.pdf,ABC Paris,0.707107
5,Gael Adebayo.pdf,,0.000000


In [16]:
# Afficher le dataframe
df_top_5[1:6]

,titre,skills,similarité
1,Jean Dupont.pdf,5 ans ABC Paris,0.866025
2,Alice Dupont.pdf,ABC Paris,0.707107
3,Pierre Martin.pdf,Bank Paris,0.707107
4,Sarah Smith.pdf,ABC Paris,0.707107
5,Gael Adebayo.pdf,,0.000000
